<a href="https://colab.research.google.com/github/groupbootcamp2/model/blob/master/AnnaModel_25_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
import numpy as np

In [ ]:
# load train and test dataset
def load_dataset():
  from google.colab import drive
  drive.mount('/content/drive')
  
  loaded_data = np.load('/content/drive/MyDrive/cfar10_modified_1000.npz')
  x_train=loaded_data['train']
  x_validation=loaded_data['validation']
  x_test=loaded_data['test']
  y_train=loaded_data['ytrain']
  y_test=loaded_data['ytest']
  y_validation=loaded_data['yvalidation']
  num_classes =np.max(y_train)+1
  # Convert class vectors to binary class matrices. This is called one hot encoding.
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  y_validation = to_categorical(y_validation)
  return x_train,y_train,x_validation,y_validation,x_test,y_test

In [ ]:

def prep_pixels(x_train,y_train,x_validation,y_validation,x_test,y_test):
  # x_train,y_train,x_validation,y_validation,x_test,y_test=load_dataset()
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_validation = x_validation.astype('float32')

  x_train /= 255
  x_test /= 255
  x_validation/=255

  return x_train,y_train,x_validation,y_validation,x_test,y_test

In [ ]:
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(15, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [ ]:
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')


In [ ]:
# load dataset
x_train,y_train,x_validation,y_validation,x_test,y_test = load_dataset()


In [ ]:
# prepare pixel data
x_train,y_train,x_validation,y_validation,x_test,y_test = prep_pixels(x_train,y_train,x_validation,y_validation,x_test,y_test)


In [ ]:
# define model
model = define_model()


In [ ]:
# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)


In [ ]:
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=32)


In [ ]:
# fit model
steps = int(x_train.shape[0] / 32)
history = model.fit_generator(it_train, steps_per_epoch=steps, epochs=400, validation_data=(x_validation,y_validation), verbose=1)


In [ ]:
# evaluate model
_, acc = model.evaluate(x_validation,y_validation, verbose=1)
print('> %.3f' % (acc * 100.0))


In [ ]:
# learning curves
summarize_diagnostics(history)